# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
query="CREATE KEYSPACE sparklyfy WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '1'};"
session.execute(query)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.execute('use sparklyfy')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
The Question 1 expects Name of the artist, title of the song and length of the track based on sessionId and itemInSession.
As we are working with a NoSQL database, we need to think about the query first which will be used to fetch the data based on which we will create the Table required.

1) The expected output is : "Name of the artist, title of the song and length of the track"
2) Based on : "sessionId and itemInSession"

From the above two points can create a table song_data:

In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

CreateTableQuery='CREATE TABLE IF NOT EXISTS song_data( sessionid int,iteminsession int,artist text,song text,length float,PRIMARY KEY (sessionid,iteminsession))'
session.execute(CreateTableQuery)


                    

Reading data from From csv file and Inserting into table

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_data (sessionid,iteminsession,artist,song,length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

#Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
query = " Select artist,song,length from song_data "
query = query + "WHERE sessionid=338 and iteminsession=4"
result=session.execute(query)
for row in result:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The Question 2 expects Name of the artist, title of the song sorted with itemInSession and users's first and last name.
We need to filter based on userid and sessionid with itemInSession as a clustering field.

1) The expected output is : "Name of the artist, title of the song , User's first name and User's last name"
2) Based on : "Userid and Sessionid"
3) Clustering column : "ItemInSession"

From the above three points can create a table song_data:

In [11]:
CreateTableQuery='CREATE TABLE IF NOT EXISTS artist_data( sessionid int,userid int,itemInSession int,artist text,song text,user_firstname text,user_lastname text ,PRIMARY KEY ((userid,sessionid),itemInSession))'
session.execute(CreateTableQuery)      

Reading data from From csv file and Inserting into table

In [12]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_data (sessionid ,userid,itemInSession,artist ,song ,user_firstname ,user_lastname ) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(line[8]),int(line[10]),int(line[3]),line[0],line[9],line[1],line[4]))
        except Exception as e:
            print(e)

Perform SELECT operation for to retreive data from the table

In [13]:
            
query = " Select artist,song,user_firstname,user_lastname from artist_data "
query = query + "WHERE userid=10 and sessionid=182"
result=session.execute(query)
for row in result:
    print (row.artist, row.song, row.user_firstname,row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The Question 3 is a direct one. It expects the users's first and last name for a particular song. Each user is assigned a unique identifier called Userid which can be used to retreive the desired output.

1) The expected output is : "User's first name and User's last name"
2) Based on : "title of the Song and Userid"

From the above three points can create a table song_data:

In [14]:
CreateTableQuery='CREATE TABLE IF NOT EXISTS user_data (song text,userid int,user_firstname text,user_lastname text,PRIMARY KEY (song,userid))'
session.execute(CreateTableQuery)

Reading data from From csv file and Inserting into table

In [15]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_data (song,userid,user_firstname,user_lastname)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

Perform SELECT operation for to retreive data from the table

In [16]:
query = " SELECT user_firstname, user_lastname FROM user_data "
query = query + "WHERE song= 'All Hands Against His Own'"
result=session.execute(query)
for row in result:
    print (row.user_firstname,row.user_lastname)  

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions

In [18]:
drop_table = "DROP TABLE song_data "  
session.execute(drop_table)

drop_table = "DROP TABLE artist_data "  
session.execute(drop_table)

drop_table = "DROP TABLE user_data "  
session.execute(drop_table)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()